# Coleta limpa: 24 B3 (.SA) + 7 indicadores (2012-01-01 ↔ 2025-09-20)

- Persistência: Parquets individuais por série em `00_data/01_bruto`.
- Janela: do mais recente (2025-09-20) até o mais antigo, cortando em 2012-01-01.
- Fontes:
  - Ações B3: investpy (com fallback yfinance para listagens mais recentes)
  - Indicadores/ETF/commodities: yfinance


In [1]:
# 1) Instalação e imports essenciais
%pip install --quiet investpy==1.0.8 yfinance==0.2.58 lxml==4.9.3 html5lib==1.1 tqdm>=4.66.0 pyarrow>=14.0.0

import os, time
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd
from tqdm import tqdm
from zoneinfo import ZoneInfo
import investpy, yfinance as yf

TZ_SP = ZoneInfo("America/Sao_Paulo")
CUT_MAX = datetime(2025, 9, 20, tzinfo=TZ_SP).date()   # mais recente
CUT_MIN = datetime(2012, 1, 1, tzinfo=TZ_SP).date()    # mais antigo
OUT_DIR = Path(r"G:/Drives compartilhados/BOLSA_2026/a_bolsa2026_gemini/00_data/01_bruto")
OUT_DIR.mkdir(parents=True, exist_ok=True)
print("OUT_DIR =", OUT_DIR)



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


g:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\.conda\Lib\site-packages\investpy\bonds.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


OUT_DIR = G:\Drives compartilhados\BOLSA_2026\a_bolsa2026_gemini\00_data\01_bruto


In [2]:
# 2) Lista dos 24 B3 (.SA) e 7 indicadores
TICKERS_B3 = [
    "ABEV3.SA", "B3SA3.SA", "BBAS3.SA", "CSNA3.SA", "CPLE6.SA", "ELET3.SA", "GGBR4.SA",
    "HAPV3.SA", "ITUB4.SA", "LREN3.SA", "PETR4.SA", "PRIO3.SA", "PSSA3.SA", "RAIL3.SA",
    "RDOR3.SA", "SBSP3.SA", "SUZB3.SA", "TAEE11.SA", "TIMS3.SA", "UGPA3.SA", "VALE3.SA",
    "VIVT3.SA", "WEGE3.SA", "TOTS3.SA"
]
INDICATORS = {
    "^BVSP": "_bvsp",
    "EWZ": "ewz",
    "^GSPC": "_gspc",
    "^VIX": "_vix",
    "DX-Y.NYB": "dx-y.nyb",
    "^TNX": "_tnx",
    "BZ=F": "bz=f",
}
print(len(TICKERS_B3), "B3 +", len(INDICATORS), "indicadores")

24 B3 + 7 indicadores


In [5]:
# 3) Utilitários de datas/normalização e coletores (investpy + yfinance)

def sp_date(d: datetime | str):
    if isinstance(d, str):
        return datetime.fromisoformat(d).date()
    return d.date() if isinstance(d, datetime) else d

def clamp_window(min_str="2012-01-01", max_str="2025-09-20"):
    dmin = datetime.fromisoformat(min_str).date()
    dmax = datetime.fromisoformat(max_str).date()
    return dmin, dmax

CUT_MIN, CUT_MAX = clamp_window("2012-01-01", "2025-09-20")

def to_investing_symbol(b3_symbol: str) -> str:
    return b3_symbol.upper().replace('.SA','')


def _to_dtindex_sp(values) -> pd.DatetimeIndex:
    idx = pd.DatetimeIndex(pd.to_datetime(values))
    if idx.tz is None:
        idx = idx.tz_localize(TZ_SP)
    else:
        idx = idx.tz_convert(TZ_SP)
    return idx


def normalize_df(df: pd.DataFrame, ticker_label: str) -> pd.DataFrame:
    rename = {"Date":"date","Open":"open","High":"high","Low":"low","Close":"close","Adj Close":"adj_close","Volume":"volume"}
    df = df.rename(columns=rename)
    # construir datetime_sp em TZ_SP
    if 'date' in df.columns:
        dt_idx = _to_dtindex_sp(df['date'])
    elif 'Date' in df.columns:
        dt_idx = _to_dtindex_sp(df['Date'])
    else:
        dt_idx = _to_dtindex_sp(df.index)
    out = df.reset_index(drop=True)
    ts = pd.Series(dt_idx)
    out['datetime_sp'] = ts.values
    out['date'] = pd.to_datetime(ts).dt.date.astype(str)
    out['ticker'] = ticker_label
    # recorte da janela
    out = out[(out['date'] >= CUT_MIN.strftime('%Y-%m-%d')) & (out['date'] <= CUT_MAX.strftime('%Y-%m-%d'))]
    # garantir colunas essenciais
    for c in ["open","high","low","close","volume"]:
        if c not in out.columns:
            out[c] = pd.NA
    out = out[["ticker","date","open","high","low","close","volume","datetime_sp"]]
    out.drop_duplicates(subset=["ticker","date"], inplace=True)
    return out


def fetch_b3(b3_symbol: str) -> pd.DataFrame:
    """Tenta investpy; se falhar, cai para yfinance (útil para TIMS3/RDOR3)."""
    sym = to_investing_symbol(b3_symbol)
    f_str = CUT_MIN.strftime('%d/%m/%Y'); t_str = CUT_MAX.strftime('%d/%m/%Y')
    try:
        # tenta via search_quotes
        res = investpy.search_quotes(text=sym, products=['stocks'], countries=['brazil'])
        qlist = res if isinstance(res, list) else ([res] if res else [])
        if qlist:
            pick = next((q for q in qlist if getattr(q,'symbol','').upper()==sym.upper()), qlist[0])
            df = pick.retrieve_historical_data(from_date=f_str, to_date=t_str, as_json=False, order='descending')
            if df is not None and not df.empty:
                return normalize_df(df, b3_symbol.upper())
        # fallback investpy direto
        df = investpy.get_stock_historical_data(stock=sym, country='brazil', from_date=f_str, to_date=t_str, as_json=False, order='descending')
        if df is not None and not df.empty:
            return normalize_df(df, b3_symbol.upper())
    except Exception:
        pass
    # yfinance fallback
    tkr = yf.Ticker(b3_symbol)
    df = tkr.history(start=CUT_MIN.strftime('%Y-%m-%d'), end=(CUT_MAX + timedelta(days=1)).strftime('%Y-%m-%d'), interval='1d', auto_adjust=False)
    if df is None or df.empty:
        raise RuntimeError(f"Sem dados para {b3_symbol}")
    return normalize_df(df, b3_symbol.upper())


def fetch_indicator(yf_symbol: str, db_symbol: str) -> pd.DataFrame:
    tkr = yf.Ticker(yf_symbol)
    df = tkr.history(start=CUT_MIN.strftime('%Y-%m-%d'), end=(CUT_MAX + timedelta(days=1)).strftime('%Y-%m-%d'), interval='1d', auto_adjust=False)
    if df is None or df.empty:
        raise RuntimeError(f"Sem dados para {yf_symbol}")
    return normalize_df(df, db_symbol)


def save_parquet(df: pd.DataFrame, out_path: Path):
    if df is None or df.empty:
        raise ValueError("DF vazio.")
    out_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_parquet(out_path, index=False)
    return out_path

In [6]:
# 4) Coleta B3 (24 .SA) — do mais recente (20/09/2025) ao mais antigo (>= 01/01/2012)
results = []
errors = []
for tk in tqdm(TICKERS_B3, desc='B3 .SA'):
    try:
        df = fetch_b3(tk)
        out = OUT_DIR / f"{tk.replace('.','_').lower()}_1d.parquet"
        save_parquet(df, out)
        results.append((tk, len(df), str(out)))
    except Exception as e:
        errors.append((tk, str(e)))
        time.sleep(0.5)

print('B3 OK:', len(results), 'Falhas:', len(errors))
results[:5], errors[:5]

B3 .SA: 100%|██████████| 24/24 [00:15<00:00,  1.56it/s]

B3 OK: 24 Falhas: 0


([('ABEV3.SA',
   3409,
   'G:\\Drives compartilhados\\BOLSA_2026\\a_bolsa2026_gemini\\00_data\\01_bruto\\abev3_sa_1d.parquet'),
  ('B3SA3.SA',
   3409,
   'G:\\Drives compartilhados\\BOLSA_2026\\a_bolsa2026_gemini\\00_data\\01_bruto\\b3sa3_sa_1d.parquet'),
  ('BBAS3.SA',
   3409,
   'G:\\Drives compartilhados\\BOLSA_2026\\a_bolsa2026_gemini\\00_data\\01_bruto\\bbas3_sa_1d.parquet'),
  ('CSNA3.SA',
   3409,
   'G:\\Drives compartilhados\\BOLSA_2026\\a_bolsa2026_gemini\\00_data\\01_bruto\\csna3_sa_1d.parquet'),
  ('CPLE6.SA',
   3408,
   'G:\\Drives compartilhados\\BOLSA_2026\\a_bolsa2026_gemini\\00_data\\01_bruto\\cple6_sa_1d.parquet')],
 [])

In [7]:
# 5) Coleta indicadores/ETF/commodities (7) via yfinance
res_i = []
err_i = []
for yf_sym, db_sym in tqdm(list(INDICATORS.items()), desc='Indicadores'):
    try:
        df = fetch_indicator(yf_sym, db_sym)
        out = OUT_DIR / f"{db_sym.replace('.','_').lower()}_1d.parquet"
        save_parquet(df, out)
        res_i.append((yf_sym, db_sym, len(df), str(out)))
    except Exception as e:
        err_i.append((yf_sym, str(e)))
        time.sleep(0.5)

print('Indicadores OK:', len(res_i), 'Falhas:', len(err_i))
res_i[:5], err_i[:5]

Indicadores: 100%|██████████| 7/7 [00:03<00:00,  1.95it/s]

Indicadores OK: 7 Falhas: 0


([('^BVSP',
   '_bvsp',
   3400,
   'G:\\Drives compartilhados\\BOLSA_2026\\a_bolsa2026_gemini\\00_data\\01_bruto\\_bvsp_1d.parquet'),
  ('EWZ',
   'ewz',
   3449,
   'G:\\Drives compartilhados\\BOLSA_2026\\a_bolsa2026_gemini\\00_data\\01_bruto\\ewz_1d.parquet'),
  ('^GSPC',
   '_gspc',
   3449,
   'G:\\Drives compartilhados\\BOLSA_2026\\a_bolsa2026_gemini\\00_data\\01_bruto\\_gspc_1d.parquet'),
  ('^VIX',
   '_vix',
   3449,
   'G:\\Drives compartilhados\\BOLSA_2026\\a_bolsa2026_gemini\\00_data\\01_bruto\\_vix_1d.parquet'),
  ('DX-Y.NYB',
   'dx-y.nyb',
   3450,
   'G:\\Drives compartilhados\\BOLSA_2026\\a_bolsa2026_gemini\\00_data\\01_bruto\\dx-y_nyb_1d.parquet')],
 [])